In [35]:
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import scipy.stats as st

In [12]:
train_rossman = pd.read_csv('/Users/cmcnamara/Downloads/Kaggle Rossman/train.csv')
store_rossman = pd.read_csv('/Users/cmcnamara/Downloads/Kaggle Rossman/store.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
train_rossman = pd.merge(train_rossman, store_rossman, on='Store', how='left').fillna(0)

In [14]:
train_rossman['StateHoliday'] = train_rossman['StateHoliday'].astype('category').cat.codes
train_rossman['StoreType'] = train_rossman['StoreType'].astype('category').cat.codes
train_rossman['Assortment'] = train_rossman['Assortment'].astype('category').cat.codes
train_rossman['Promo2SinceWeek'] = train_rossman['Promo2SinceWeek'].astype(int)
train_rossman['Promo2SinceYear'] = train_rossman['Promo2SinceYear'].astype(int)
train_rossman['CompeititonDistance'] = train_rossman['CompetitionDistance'].astype(int)
train_rossman['CompetitionOpenSinceMonth'] = train_rossman['CompetitionOpenSinceMonth'].astype(int)
train_rossman['CompetitionOpenSinceYear'] = train_rossman["CompetitionOpenSinceYear"].astype(int)
train_rossman['PromoInterval'] = train_rossman['PromoInterval'].astype('category').cat.codes
train_rossman['Date'] = pd.to_datetime(train_rossman['Date'])

In [15]:
test_rossman = train_rossman.iloc[np.where(train_rossman.Date.dt.year == 2015)[0], :]
train_rossman = train_rossman.iloc[np.where(train_rossman.Date.dt.year == 2014)[0], :]

In [16]:
train_rossman = train_rossman.iloc[::-1]
test_rossman = test_rossman.iloc[::-1]

Note that there are a few problems when comparing the different algorithms (xgboost, lightgbm, catboost).

* The first is that they take the input data in differently. While  xgboost and lightgbm have wrappers to handle pandas dataframes directly, catboost does not. Therefore, I will use their native data wrapper for each algorithm.
* The second is that they do not grow the trees the same.

Nonetheless, I will try to ensure that each algorithm is as close to one another as possible for maximum accuracy."

In [23]:
train_y = train_rossman['Sales']
train_x = train_rossman.drop(['Sales', 'Date'], axis=1)
params = {"max_depth": [5,10, 15],
         "learning_rate": [0.01, 0.05, 0.1],
         "n_estimators": [100, 250]}
params_cb = {'depth': [5, 10, 15],
            'learning_rate': [0.01, 0.05, 0.1],
            'iterations': [100, 250]}
test_y = test_rossman['Sales']
test_x = test_rossman.drop(['Sales', 'Date'], axis=1)
n_splits = 3
max_train_size = len(train_x) // (n_splits+1)

In [18]:
train_x.shape

(373855, 17)

In [19]:
test_x.shape

(236380, 17)

# Grid Search

### xgboost

In [31]:
%%time
model_xg = xgb.XGBRegressor()
grid_search_xg = GridSearchCV(model_xg, param_grid=params, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), verbose=10, n_jobs=1)
grid_search_xg.fit(train_x, train_y)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.4741894480519151, total=   5.9s
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.474585362689519, total=   5.9s
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.3s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.40926535212583226, total=   6.0s
[CV] learning_rate=0.01, max_depth=5, n_estimators=250 ...............


...

[CV]  learning_rate=0.01, max_depth=10, n_estimators=100, score=0.45032431807232465, total=  14.6s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.9min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9342543296817523, total=  37.0s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9387704383121537, total=  36.8s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9077792163819632, total=  37.4s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.5194970143939951, total=  21.8s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.4961814705948933, total=  21.6s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.4545445045046473, total=  21.8s
[CV] learning_rate=0.01, max_depth=15, n_estimators=250 .

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 26.3min finished


CPU times: user 28min 54s, sys: 6.11 s, total: 29min
Wall time: 29min 5s


### lightgbm

In [29]:
%%time
model_lg = lgbm.LGBMRegressor()
grid_search_lg = GridSearchCV(model_lg, param_grid=params, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), verbose=10, n_jobs=1)
grid_search_lg.fit(train_x, train_y)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............
[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.7702084297522601, total=   1.9s
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.7802940154265927, total=   1.8s
[CV] learning_rate=0.01, max_depth=5, n_estimators=100 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.7s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, n_estimators=100, score=0.7411855579601883, total=   1.7s
[CV] learning_rate=0.01, max_depth=5, n_estimators=250 ...............


...

[CV]  learning_rate=0.01, max_depth=10, n_estimators=100, score=0.7525415953023233, total=   1.9s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   30.5s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9281191923437121, total=   4.6s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9258024743785994, total=   4.5s
[CV] learning_rate=0.01, max_depth=10, n_estimators=250 ..............
[CV]  learning_rate=0.01, max_depth=10, n_estimators=250, score=0.9086480819570032, total=   4.6s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.7807000046381917, total=   1.9s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.7889879656053237, total=   1.9s
[CV] learning_rate=0.01, max_depth=15, n_estimators=100 ..............
[CV]  learning_rate=0.01, max_depth=15, n_estimators=100, score=0.7525415953023233, total=   1.9s
[CV] learning_rate=0.01, max_depth=15, n_estimators=250 .

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  3.5min finished


CPU times: user 3min 34s, sys: 3.14 s, total: 3min 37s
Wall time: 3min 38s


### catboost

In [30]:
%%time
model_cb = cb.CatBoostRegressor(verbose=False)
grid_search_cb = GridSearchCV(model_cb, param_grid=params_cb, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), verbose=10, n_jobs=1)
grid_search_cb.fit(train_x, train_y)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] depth=5, iterations=100, learning_rate=0.01 .....................
[CV]  depth=5, iterations=100, learning_rate=0.01, score=3009.6039797173053, total=   3.6s
[CV] depth=5, iterations=100, learning_rate=0.01 .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV]  depth=5, iterations=100, learning_rate=0.01, score=2748.4276349439524, total=   3.5s
[CV] depth=5, iterations=100, learning_rate=0.01 .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.3s remaining:    0.0s


[CV]  depth=5, iterations=100, learning_rate=0.01, score=3345.7140785532606, total=   3.4s
[CV] depth=5, iterations=100, learning_rate=0.05 .....................


...

[CV]  depth=5, iterations=100, learning_rate=0.1, score=1110.6048711253784, total=   3.4s
[CV] depth=5, iterations=250, learning_rate=0.01 .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   37.1s remaining:    0.0s


[CV]  depth=5, iterations=250, learning_rate=0.01, score=1430.1639514556234, total=   7.0s
[CV] depth=5, iterations=250, learning_rate=0.01 .....................
[CV]  depth=5, iterations=250, learning_rate=0.01, score=1244.5904044273348, total=   6.5s
[CV] depth=5, iterations=250, learning_rate=0.01 .....................
[CV]  depth=5, iterations=250, learning_rate=0.01, score=1650.824371940917, total=   7.0s
[CV] depth=5, iterations=250, learning_rate=0.05 .....................
[CV]  depth=5, iterations=250, learning_rate=0.05, score=926.6333496032473, total=   8.4s
[CV] depth=5, iterations=250, learning_rate=0.05 .....................
[CV]  depth=5, iterations=250, learning_rate=0.05, score=898.2463130211527, total=   6.8s
[CV] depth=5, iterations=250, learning_rate=0.05 .....................
[CV]  depth=5, iterations=250, learning_rate=0.05, score=1081.560587869713, total=   6.5s
[CV] depth=5, iterations=250, learning_rate=0.1 ......................
[CV]  depth=5, iterations=250, l

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 24.6min finished


CPU times: user 49min 44s, sys: 20min 13s, total: 1h 9min 58s
Wall time: 24min 44s


## Predictions

In [33]:
predict_y_xg = grid_search_xg.predict(test_x)
predict_y_lg = grid_search_lg.predict(test_x)
predict_y_cb = grid_search_cb.predict(test_x)

print('XGBoost R2 Score: {}'.format(r2_score(test_y, predict_y_xg)))
print('LightGBM R2 Score: {}'.format(r2_score(test_y, predict_y_lg)))
print('CatBoost R2 Score: {}'.format(r2_score(test_y, predict_y_cb)))

print('XGBoost MSE Score: {}'.format(mean_squared_error(test_y, predict_y_xg)))
print('LightGBM MSE Score: {}'.format(mean_squared_error(test_y, predict_y_lg)))
print('CatBoost MSE Score: {}'.format(mean_squared_error(test_y, predict_y_cb)))

print('XGBoost MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_xg)))
print('LightGBM MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_lg)))
print('CatBoost MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_cb)))

XGBoost R2 Score: 0.9845042161602857
LightGBM R2 Score: 0.9593680458739346
CatBoost R2 Score: 0.3920928256889047
XGBoost MSE Score: 228517.11513249998
LightGBM MSE Score: 599201.5012036816
CatBoost MSE Score: 8964838.124928502
XGBoost MAE Score: 304.35978959594553
LightGBM MAE Score: 524.8717335935854
CatBoost MAE Score: 2273.944188598435


# Randomized Search

In [44]:
params = {'max_depth': st.randint(3, 16),
          'learning_rate': st.uniform(0.01, 0.4),
          'n_estimators': st.randint(100, 500)
}

params_cb = {'depth': st.randint(3,16),
            'learning_rate': st.uniform(0.01, 0.4),
            'iterations': st.randint(100, 500)}

### xgboost

In [46]:
%%time
model_xg = xgb.XGBRegressor()
rand_search_xg = RandomizedSearchCV(model_xg, params, n_jobs=1, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), n_iter=3, verbose=10)
rand_search_xg.fit(train_x, train_y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] learning_rate=0.1189851975776845, max_depth=5, n_estimators=456 .
[CV]  learning_rate=0.1189851975776845, max_depth=5, n_estimators=456, score=0.9812714919490011, total=  27.0s
[CV] learning_rate=0.1189851975776845, max_depth=5, n_estimators=456 .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.1s remaining:    0.0s


[CV]  learning_rate=0.1189851975776845, max_depth=5, n_estimators=456, score=0.9796712250336942, total=  27.0s
[CV] learning_rate=0.1189851975776845, max_depth=5, n_estimators=456 .


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   58.1s remaining:    0.0s


[CV]  learning_rate=0.1189851975776845, max_depth=5, n_estimators=456, score=0.9688285170181603, total=  27.0s
[CV] learning_rate=0.23753539804279403, max_depth=6, n_estimators=121 


...

[CV]  learning_rate=0.3271156452225856, max_depth=13, n_estimators=200, score=0.9802210781305453, total=  36.6s
[CV] learning_rate=0.3271156452225856, max_depth=13, n_estimators=200 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  3.3min remaining:    0.0s


[CV]  learning_rate=0.3271156452225856, max_depth=13, n_estimators=200, score=0.9643473226615725, total=  36.2s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.9min finished


CPU times: user 5min 46s, sys: 844 ms, total: 5min 47s
Wall time: 5min 47s


### lightgbm

In [39]:
%%time
model_lg = lgbm.LGBMRegressor()
rand_search_lg = RandomizedSearchCV(model_lg, params, n_jobs=1, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), n_iter=3, verbose=10)
rand_search_lg.fit(train_x, train_y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] learning_rate=0.23071942288924802, max_depth=12, n_estimators=378 
[CV]  learning_rate=0.23071942288924802, max_depth=12, n_estimators=378, score=0.9807253556890937, total=   5.1s
[CV] learning_rate=0.23071942288924802, max_depth=12, n_estimators=378 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s remaining:    0.0s


[CV]  learning_rate=0.23071942288924802, max_depth=12, n_estimators=378, score=0.9789644476003548, total=   4.9s
[CV] learning_rate=0.23071942288924802, max_depth=12, n_estimators=378 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.3s remaining:    0.0s


[CV]  learning_rate=0.23071942288924802, max_depth=12, n_estimators=378, score=0.9678459475699133, total=   4.9s
[CV] learning_rate=0.11612205537040937, max_depth=7, n_estimators=318 


...

[CV]  learning_rate=0.08764611307822062, max_depth=4, n_estimators=231, score=0.9375465864178051, total=   2.6s
[CV] learning_rate=0.08764611307822062, max_depth=4, n_estimators=231 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   48.0s remaining:    0.0s


[CV]  learning_rate=0.08764611307822062, max_depth=4, n_estimators=231, score=0.9338356824147633, total=   2.6s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   51.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   51.7s finished


CPU times: user 1min 1s, sys: 591 ms, total: 1min 2s
Wall time: 1min 2s


### catboost

In [45]:
%%time
model_cb = cb.CatBoostRegressor()
rand_search_cb = RandomizedSearchCV(model_cb, params_cb, n_jobs=1, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), n_iter=3, verbose=10)
rand_search_cb.fit(train_x, train_y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] depth=3, iterations=277, learning_rate=0.07967928664484071 ......
0:	learn: 6148.1391902	total: 13.3ms	remaining: 3.66s
1:	learn: 5702.8508254	total: 25.8ms	remaining: 3.54s
2:	learn: 5292.8626967	total: 37.9ms	remaining: 3.46s
3:	learn: 4916.7826641	total: 49.4ms	remaining: 3.37s
4:	learn: 4571.0473659	total: 60.8ms	remaining: 3.31s
5:	learn: 4259.0660003	total: 72.9ms	remaining: 3.29s
6:	learn: 3971.1421736	total: 84.9ms	remaining: 3.27s
7:	learn: 3707.4290081	total: 96.9ms	remaining: 3.26s
8:	learn: 3468.6489978	total: 110ms	remaining: 3.28s
9:	learn: 3250.1466209	total: 127ms	remaining: 3.38s
10:	learn: 3049.6270557	total: 144ms	remaining: 3.49s
11:	learn: 2869.0878050	total: 159ms	remaining: 3.52s
12:	learn: 2702.1551283	total: 171ms	remaining: 3.48s
13:	learn: 2549.9268699	total: 184ms	remaining: 3.45s
14:	learn: 2412.6325603	total: 196ms	remaining: 3.42s
15:	learn: 2289.0189154	total: 208ms	remaining: 3.39s
16:	lear

162:	learn: 917.3323461	total: 2.31s	remaining: 1.61s
163:	learn: 916.6735467	total: 2.33s	remaining: 1.6s
164:	learn: 915.9012242	total: 2.34s	remaining: 1.59s
165:	learn: 915.4007836	total: 2.35s	remaining: 1.57s
166:	learn: 915.0445398	total: 2.36s	remaining: 1.56s
167:	learn: 914.4310768	total: 2.38s	remaining: 1.54s
168:	learn: 913.9414128	total: 2.39s	remaining: 1.53s
169:	learn: 913.5436710	total: 2.4s	remaining: 1.51s
170:	learn: 912.9882037	total: 2.41s	remaining: 1.5s
171:	learn: 912.3186004	total: 2.42s	remaining: 1.48s
172:	learn: 911.5601313	total: 2.44s	remaining: 1.47s
173:	learn: 911.0808725	total: 2.45s	remaining: 1.45s
174:	learn: 910.4286298	total: 2.46s	remaining: 1.44s
175:	learn: 909.9098937	total: 2.48s	remaining: 1.42s
176:	learn: 909.1654377	total: 2.49s	remaining: 1.41s
177:	learn: 908.5945406	total: 2.5s	remaining: 1.39s
178:	learn: 908.0210318	total: 2.52s	remaining: 1.38s
179:	learn: 907.5336679	total: 2.54s	remaining: 1.37s
180:	learn: 907.0861340	total: 2

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.4s remaining:    0.0s


0:	learn: 6465.1304069	total: 14.5ms	remaining: 3.99s
1:	learn: 5996.9231563	total: 27.9ms	remaining: 3.84s
2:	learn: 5567.4478955	total: 41.5ms	remaining: 3.79s
3:	learn: 5173.5823685	total: 54.8ms	remaining: 3.74s
4:	learn: 4812.7502671	total: 67.9ms	remaining: 3.69s
5:	learn: 4484.3586039	total: 81.2ms	remaining: 3.67s
6:	learn: 4180.8918806	total: 94.6ms	remaining: 3.65s
7:	learn: 3903.3197420	total: 108ms	remaining: 3.63s
8:	learn: 3653.2229550	total: 121ms	remaining: 3.59s
9:	learn: 3422.8114550	total: 133ms	remaining: 3.56s
10:	learn: 3212.3349048	total: 147ms	remaining: 3.54s
11:	learn: 3025.1626383	total: 160ms	remaining: 3.54s
12:	learn: 2851.3793527	total: 174ms	remaining: 3.53s
13:	learn: 2694.2993082	total: 187ms	remaining: 3.51s
14:	learn: 2551.8298213	total: 200ms	remaining: 3.49s
15:	learn: 2423.4007637	total: 214ms	remaining: 3.5s
16:	learn: 2306.0814001	total: 233ms	remaining: 3.56s
17:	learn: 2201.2763144	total: 255ms	remaining: 3.67s
18:	learn: 2103.2333913	total: 2

160:	learn: 995.1195631	total: 2.29s	remaining: 1.65s
161:	learn: 994.5392700	total: 2.31s	remaining: 1.64s
162:	learn: 993.9478199	total: 2.33s	remaining: 1.63s
163:	learn: 993.2657383	total: 2.35s	remaining: 1.62s
164:	learn: 992.8102867	total: 2.36s	remaining: 1.6s
165:	learn: 991.6781625	total: 2.38s	remaining: 1.59s
166:	learn: 991.1867579	total: 2.39s	remaining: 1.58s
167:	learn: 990.3989312	total: 2.41s	remaining: 1.56s
168:	learn: 989.6080664	total: 2.43s	remaining: 1.55s
169:	learn: 989.0910245	total: 2.44s	remaining: 1.54s
170:	learn: 987.7678191	total: 2.46s	remaining: 1.52s
171:	learn: 987.1123584	total: 2.48s	remaining: 1.51s
172:	learn: 986.6634343	total: 2.5s	remaining: 1.5s
173:	learn: 986.1771799	total: 2.51s	remaining: 1.49s
174:	learn: 985.5214578	total: 2.53s	remaining: 1.48s
175:	learn: 985.0732575	total: 2.55s	remaining: 1.46s
176:	learn: 984.2643640	total: 2.56s	remaining: 1.45s
177:	learn: 983.7529995	total: 2.58s	remaining: 1.44s
178:	learn: 983.2643695	total: 

...

0:	learn: 5313.6865682	total: 29.7ms	remaining: 7.39s
1:	learn: 4136.5515789	total: 56.4ms	remaining: 6.99s
2:	learn: 3260.8661145	total: 84.3ms	remaining: 6.94s
3:	learn: 2609.2247314	total: 112ms	remaining: 6.89s
4:	learn: 2130.0071310	total: 138ms	remaining: 6.77s
5:	learn: 1784.1448353	total: 166ms	remaining: 6.76s
6:	learn: 1533.5462534	total: 193ms	remaining: 6.69s
7:	learn: 1357.1053643	total: 219ms	remaining: 6.64s
8:	learn: 1236.9397864	total: 252ms	remaining: 6.76s
9:	learn: 1158.2810758	total: 291ms	remaining: 6.98s
10:	learn: 1091.8153693	total: 316ms	remaining: 6.86s
11:	learn: 1047.9669419	total: 340ms	remaining: 6.74s
12:	learn: 1016.9152830	total: 365ms	remaining: 6.66s
13:	learn: 996.8316288	total: 391ms	remaining: 6.59s
14:	learn: 979.4780748	total: 417ms	remaining: 6.54s
15:	learn: 968.8970388	total: 446ms	remaining: 6.53s
16:	learn: 953.9185772	total: 483ms	remaining: 6.63s
17:	learn: 942.1521260	total: 510ms	remaining: 6.58s
18:	learn: 932.6458177	total: 536ms	rema

160:	learn: 553.5942080	total: 4.35s	remaining: 2.41s
161:	learn: 551.6852225	total: 4.39s	remaining: 2.38s
162:	learn: 551.3414780	total: 4.42s	remaining: 2.36s
163:	learn: 549.6142826	total: 4.46s	remaining: 2.34s
164:	learn: 548.4727205	total: 4.48s	remaining: 2.31s
165:	learn: 547.7291781	total: 4.51s	remaining: 2.28s
166:	learn: 546.8563041	total: 4.53s	remaining: 2.25s
167:	learn: 545.7023970	total: 4.56s	remaining: 2.23s
168:	learn: 544.3865444	total: 4.6s	remaining: 2.2s
169:	learn: 544.0047950	total: 4.62s	remaining: 2.18s
170:	learn: 542.4166401	total: 4.65s	remaining: 2.15s
171:	learn: 542.0018691	total: 4.68s	remaining: 2.12s
172:	learn: 541.1174597	total: 4.7s	remaining: 2.09s
173:	learn: 539.3930399	total: 4.73s	remaining: 2.06s
174:	learn: 538.4085778	total: 4.75s	remaining: 2.04s
175:	learn: 537.7970085	total: 4.79s	remaining: 2.01s
176:	learn: 537.4437977	total: 4.81s	remaining: 1.98s
177:	learn: 536.9523928	total: 4.84s	remaining: 1.96s
178:	learn: 536.4110401	total: 

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  6.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  6.1min finished


0:	learn: 6497.1655534	total: 56ms	remaining: 15.5s
1:	learn: 6026.3557757	total: 110ms	remaining: 15.1s
2:	learn: 5593.2818124	total: 162ms	remaining: 14.8s
3:	learn: 5195.0211966	total: 220ms	remaining: 15s
4:	learn: 4835.0387519	total: 286ms	remaining: 15.5s
5:	learn: 4500.6717672	total: 341ms	remaining: 15.4s
6:	learn: 4197.8459710	total: 388ms	remaining: 15s
7:	learn: 3919.1629883	total: 437ms	remaining: 14.7s
8:	learn: 3668.4744739	total: 485ms	remaining: 14.4s
9:	learn: 3437.5813304	total: 543ms	remaining: 14.5s
10:	learn: 3227.1363570	total: 592ms	remaining: 14.3s
11:	learn: 3037.6592330	total: 639ms	remaining: 14.1s
12:	learn: 2863.6552105	total: 688ms	remaining: 14s
13:	learn: 2704.5225540	total: 740ms	remaining: 13.9s
14:	learn: 2561.6970440	total: 796ms	remaining: 13.9s
15:	learn: 2430.4892485	total: 845ms	remaining: 13.8s
16:	learn: 2313.2018710	total: 895ms	remaining: 13.7s
17:	learn: 2207.8789738	total: 942ms	remaining: 13.5s
18:	learn: 2110.9510568	total: 1s	remaining: 

152:	learn: 1015.0817589	total: 7.83s	remaining: 6.34s
153:	learn: 1014.7324788	total: 7.88s	remaining: 6.29s
154:	learn: 1014.1223964	total: 7.93s	remaining: 6.24s
155:	learn: 1013.3439379	total: 7.97s	remaining: 6.18s
156:	learn: 1012.7968872	total: 8.02s	remaining: 6.13s
157:	learn: 1012.2249337	total: 8.09s	remaining: 6.09s
158:	learn: 1011.7862764	total: 8.13s	remaining: 6.04s
159:	learn: 1011.3920032	total: 8.18s	remaining: 5.98s
160:	learn: 1010.6895894	total: 8.23s	remaining: 5.93s
161:	learn: 1010.1659057	total: 8.28s	remaining: 5.88s
162:	learn: 1009.5989023	total: 8.34s	remaining: 5.83s
163:	learn: 1009.1496136	total: 8.39s	remaining: 5.78s
164:	learn: 1008.6116324	total: 8.43s	remaining: 5.72s
165:	learn: 1007.9399327	total: 8.48s	remaining: 5.67s
166:	learn: 1007.5933154	total: 8.53s	remaining: 5.62s
167:	learn: 1007.2166183	total: 8.59s	remaining: 5.57s
168:	learn: 1006.8011506	total: 8.64s	remaining: 5.52s
169:	learn: 1006.3349738	total: 8.69s	remaining: 5.47s
170:	learn

In [47]:
predict_y_xg = rand_search_xg.predict(test_x)
predict_y_lg = rand_search_lg.predict(test_x)
predict_y_cb = rand_search_cb.predict(test_x)

print('XGBoost R2 Score: {}'.format(r2_score(test_y, predict_y_xg)))
print('LightGBM R2 Score: {}'.format(r2_score(test_y, predict_y_lg)))
print('CatBoost R2 Score: {}'.format(r2_score(test_y, predict_y_cb)))

print('XGBoost MSE Score: {}'.format(mean_squared_error(test_y, predict_y_xg)))
print('LightGBM MSE Score: {}'.format(mean_squared_error(test_y, predict_y_lg)))
print('CatBoost MSE Score: {}'.format(mean_squared_error(test_y, predict_y_cb)))

print('XGBoost MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_xg)))
print('LightGBM MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_lg)))
print('CatBoost MAE Score: {}'.format(mean_absolute_error(test_y, predict_y_cb)))

XGBoost R2 Score: 0.9792073740594721
LightGBM R2 Score: 0.9763045580765038
CatBoost R2 Score: 0.9330458562276951
XGBoost MSE Score: 306629.9159246819
LightGBM MSE Score: 349437.8913746444
CatBoost MSE Score: 987376.1753052367
XGBoost MAE Score: 377.52662160216727
LightGBM MAE Score: 402.459496846186
CatBoost MAE Score: 676.0156816696343


# Early Stopping

### xgboost

In [65]:
train_xg = xgb.DMatrix(train_x, train_y)
params_xg = {"objective": 'reg:linear',
             'eval_metric': 'rmse'
}
validation_xg = xgb.DMatrix(test_x, test_y)
watchlist  = [ (train_xg,'train'),(validation_xg,'eval')]

In [69]:
%%time
model_xg = xgb.train(params_xg, train_xg, num_boost_round=9999, evals=watchlist, early_stopping_rounds=10)

[0]	train-rmse:4987.38	eval-rmse:5045.6
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[1]	train-rmse:3592.67	eval-rmse:3685.76
[2]	train-rmse:2642.85	eval-rmse:2755.06
[3]	train-rmse:2005.34	eval-rmse:2127.65
[4]	train-rmse:1589.2	eval-rmse:1709.36
[5]	train-rmse:1328.17	eval-rmse:1441.95
[6]	train-rmse:1164.66	eval-rmse:1266.67
[7]	train-rmse:1068.21	eval-rmse:1156.75
[8]	train-rmse:1005.87	eval-rmse:1078.82
[9]	train-rmse:958.125	eval-rmse:1024.92
[10]	train-rmse:930.629	eval-rmse:989.363
[11]	train-rmse:913.679	eval-rmse:964.711
[12]	train-rmse:899.038	eval-rmse:947.171
[13]	train-rmse:882.93	eval-rmse:930.399
[14]	train-rmse:873.873	eval-rmse:919.367
[15]	train-rmse:856.776	eval-rmse:903.818
[16]	train-rmse:837.108	eval-rmse:884.686
[17]	train-rmse:831.494	eval-rmse:878.347
[18]	train-rmse:822.839	eval-rmse:867.61
[19]	train-rmse:814.927	eval-rmse:859.665
[20]	train-rmse:803.924	eval-rm

[191]	train-rmse:435.355	eval-rmse:511.741
[192]	train-rmse:434.838	eval-rmse:511.322
[193]	train-rmse:434.362	eval-rmse:510.938
[194]	train-rmse:434.214	eval-rmse:510.947
[195]	train-rmse:433.744	eval-rmse:510.474
[196]	train-rmse:433.644	eval-rmse:510.464
[197]	train-rmse:432.9	eval-rmse:509.753
[198]	train-rmse:432.605	eval-rmse:509.523
[199]	train-rmse:432.091	eval-rmse:509.225
[200]	train-rmse:431.75	eval-rmse:509.156
[201]	train-rmse:431.163	eval-rmse:508.763
[202]	train-rmse:430.801	eval-rmse:508.62
[203]	train-rmse:430.359	eval-rmse:508.406
[204]	train-rmse:430.217	eval-rmse:508.328
[205]	train-rmse:429.848	eval-rmse:508.08
[206]	train-rmse:429.057	eval-rmse:507.373
[207]	train-rmse:428.716	eval-rmse:507.12
[208]	train-rmse:428.381	eval-rmse:507.06
[209]	train-rmse:427.616	eval-rmse:506.624
[210]	train-rmse:427.46	eval-rmse:506.417
[211]	train-rmse:427.139	eval-rmse:506.257
[212]	train-rmse:426.645	eval-rmse:505.833
[213]	train-rmse:426.516	eval-rmse:505.783
[214]	train-rmse:42

[383]	train-rmse:385.838	eval-rmse:484.277
[384]	train-rmse:385.616	eval-rmse:484.217
[385]	train-rmse:385.476	eval-rmse:484.158
[386]	train-rmse:385.195	eval-rmse:483.941
[387]	train-rmse:385.091	eval-rmse:483.931
[388]	train-rmse:385.022	eval-rmse:483.95
[389]	train-rmse:384.784	eval-rmse:483.769
[390]	train-rmse:384.535	eval-rmse:483.599
[391]	train-rmse:384.372	eval-rmse:483.576
[392]	train-rmse:384.157	eval-rmse:483.438
[393]	train-rmse:383.892	eval-rmse:483.219
[394]	train-rmse:383.698	eval-rmse:483.148
[395]	train-rmse:383.489	eval-rmse:483.047
[396]	train-rmse:383.35	eval-rmse:482.969
[397]	train-rmse:383.092	eval-rmse:482.839
[398]	train-rmse:382.879	eval-rmse:482.861
[399]	train-rmse:382.812	eval-rmse:482.839
[400]	train-rmse:382.631	eval-rmse:482.753
[401]	train-rmse:382.516	eval-rmse:482.79
[402]	train-rmse:382.383	eval-rmse:482.759
[403]	train-rmse:382.292	eval-rmse:482.723
[404]	train-rmse:382.18	eval-rmse:482.687
[405]	train-rmse:382.054	eval-rmse:482.791
[406]	train-rms

### lightgbm

In [59]:
train_lg = lgbm.Dataset(train_x, train_y)
params_lg = {'task': 'train',
            'objective': 'regression',
            'metric': 'rmse'}
validation_lg = lgbm.Dataset(test_x, test_y, reference=train_lg)

In [62]:
%%time
model_lg = lgbm.train(params_lg, train_lg, valid_sets=validation_lg, num_boost_round=9999, early_stopping_rounds=10)

[1]	valid_0's rmse: 3771.34
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 3703.43
[3]	valid_0's rmse: 3636.29
[4]	valid_0's rmse: 3570.17
[5]	valid_0's rmse: 3504.8
[6]	valid_0's rmse: 3440.94
[7]	valid_0's rmse: 3378.38
[8]	valid_0's rmse: 3316.11
[9]	valid_0's rmse: 3255.75
[10]	valid_0's rmse: 3195.7
[11]	valid_0's rmse: 3136.86
[12]	valid_0's rmse: 3078.81
[13]	valid_0's rmse: 3021.99
[14]	valid_0's rmse: 2966.35
[15]	valid_0's rmse: 2911.27
[16]	valid_0's rmse: 2857.63
[17]	valid_0's rmse: 2804.22
[18]	valid_0's rmse: 2752.21
[19]	valid_0's rmse: 2700.11
[20]	valid_0's rmse: 2650.39
[21]	valid_0's rmse: 2599.77
[22]	valid_0's rmse: 2550.37
[23]	valid_0's rmse: 2502.53
[24]	valid_0's rmse: 2454.13
[25]	valid_0's rmse: 2406.91
[26]	valid_0's rmse: 2361.02
[27]	valid_0's rmse: 2315.82
[28]	valid_0's rmse: 2272.13
[29]	valid_0's rmse: 2227.88
[30]	valid_0's rmse: 2184.9
[31]	valid_0's rmse: 2143.24
[32]	valid_0's rmse: 2101.94
[33]	valid_0's rmse: 2

[280]	valid_0's rmse: 736.216
[281]	valid_0's rmse: 735.006
[282]	valid_0's rmse: 733.472
[283]	valid_0's rmse: 732.772
[284]	valid_0's rmse: 731.63
[285]	valid_0's rmse: 729.715
[286]	valid_0's rmse: 727.798
[287]	valid_0's rmse: 727.532
[288]	valid_0's rmse: 726.309
[289]	valid_0's rmse: 726.067
[290]	valid_0's rmse: 725.488
[291]	valid_0's rmse: 724.025
[292]	valid_0's rmse: 721.522
[293]	valid_0's rmse: 719.972
[294]	valid_0's rmse: 719.018
[295]	valid_0's rmse: 718.645
[296]	valid_0's rmse: 718.291
[297]	valid_0's rmse: 716.509
[298]	valid_0's rmse: 715.5
[299]	valid_0's rmse: 712.618
[300]	valid_0's rmse: 711.786
[301]	valid_0's rmse: 710.956
[302]	valid_0's rmse: 709.916
[303]	valid_0's rmse: 709.202
[304]	valid_0's rmse: 707.383
[305]	valid_0's rmse: 705.79
[306]	valid_0's rmse: 704.498
[307]	valid_0's rmse: 703.563
[308]	valid_0's rmse: 702.107
[309]	valid_0's rmse: 700.644
[310]	valid_0's rmse: 699.94
[311]	valid_0's rmse: 697.638
[312]	valid_0's rmse: 696.674
[313]	valid_0's

[554]	valid_0's rmse: 547.194
[555]	valid_0's rmse: 546.984
[556]	valid_0's rmse: 546.989
[557]	valid_0's rmse: 547.008
[558]	valid_0's rmse: 546.726
[559]	valid_0's rmse: 546.246
[560]	valid_0's rmse: 546.182
[561]	valid_0's rmse: 545.988
[562]	valid_0's rmse: 545.683
[563]	valid_0's rmse: 545.304
[564]	valid_0's rmse: 544.924
[565]	valid_0's rmse: 544.588
[566]	valid_0's rmse: 544.139
[567]	valid_0's rmse: 543.769
[568]	valid_0's rmse: 543.195
[569]	valid_0's rmse: 542.767
[570]	valid_0's rmse: 542.45
[571]	valid_0's rmse: 542.486
[572]	valid_0's rmse: 542.255
[573]	valid_0's rmse: 541.924
[574]	valid_0's rmse: 541.612
[575]	valid_0's rmse: 541.405
[576]	valid_0's rmse: 541.187
[577]	valid_0's rmse: 541.222
[578]	valid_0's rmse: 540.904
[579]	valid_0's rmse: 540.757
[580]	valid_0's rmse: 540.608
[581]	valid_0's rmse: 540.561
[582]	valid_0's rmse: 540.593
[583]	valid_0's rmse: 540.459
[584]	valid_0's rmse: 539.918
[585]	valid_0's rmse: 539.377
[586]	valid_0's rmse: 539.043
[587]	valid

[829]	valid_0's rmse: 503.954
[830]	valid_0's rmse: 503.903
[831]	valid_0's rmse: 503.892
[832]	valid_0's rmse: 503.752
[833]	valid_0's rmse: 503.582
[834]	valid_0's rmse: 503.47
[835]	valid_0's rmse: 503.415
[836]	valid_0's rmse: 503.373
[837]	valid_0's rmse: 503.286
[838]	valid_0's rmse: 503.159
[839]	valid_0's rmse: 503.102
[840]	valid_0's rmse: 502.904
[841]	valid_0's rmse: 502.86
[842]	valid_0's rmse: 502.71
[843]	valid_0's rmse: 502.589
[844]	valid_0's rmse: 502.494
[845]	valid_0's rmse: 502.498
[846]	valid_0's rmse: 502.246
[847]	valid_0's rmse: 502.081
[848]	valid_0's rmse: 501.979
[849]	valid_0's rmse: 501.94
[850]	valid_0's rmse: 501.828
[851]	valid_0's rmse: 501.717
[852]	valid_0's rmse: 501.694
[853]	valid_0's rmse: 501.68
[854]	valid_0's rmse: 501.624
[855]	valid_0's rmse: 501.555
[856]	valid_0's rmse: 501.41
[857]	valid_0's rmse: 501.432
[858]	valid_0's rmse: 501.384
[859]	valid_0's rmse: 501.226
[860]	valid_0's rmse: 501.151
[861]	valid_0's rmse: 501.275
[862]	valid_0's 

[1102]	valid_0's rmse: 488.485
[1103]	valid_0's rmse: 488.433
[1104]	valid_0's rmse: 488.372
[1105]	valid_0's rmse: 488.389
[1106]	valid_0's rmse: 488.379
[1107]	valid_0's rmse: 488.367
[1108]	valid_0's rmse: 488.332
[1109]	valid_0's rmse: 488.3
[1110]	valid_0's rmse: 488.25
[1111]	valid_0's rmse: 488.163
[1112]	valid_0's rmse: 488.129
[1113]	valid_0's rmse: 488.065
[1114]	valid_0's rmse: 488.016
[1115]	valid_0's rmse: 487.98
[1116]	valid_0's rmse: 487.961
[1117]	valid_0's rmse: 487.95
[1118]	valid_0's rmse: 487.959
[1119]	valid_0's rmse: 487.943
[1120]	valid_0's rmse: 487.902
[1121]	valid_0's rmse: 487.883
[1122]	valid_0's rmse: 487.808
[1123]	valid_0's rmse: 487.767
[1124]	valid_0's rmse: 487.757
[1125]	valid_0's rmse: 487.698
[1126]	valid_0's rmse: 487.692
[1127]	valid_0's rmse: 487.667
[1128]	valid_0's rmse: 487.653
[1129]	valid_0's rmse: 487.652
[1130]	valid_0's rmse: 487.618
[1131]	valid_0's rmse: 487.614
[1132]	valid_0's rmse: 487.569
[1133]	valid_0's rmse: 487.543
[1134]	valid_

[1375]	valid_0's rmse: 480.98
[1376]	valid_0's rmse: 480.956
[1377]	valid_0's rmse: 480.948
[1378]	valid_0's rmse: 481.09
[1379]	valid_0's rmse: 481.065
[1380]	valid_0's rmse: 481.054
[1381]	valid_0's rmse: 481.067
[1382]	valid_0's rmse: 481.025
[1383]	valid_0's rmse: 481.032
[1384]	valid_0's rmse: 481.006
[1385]	valid_0's rmse: 480.989
[1386]	valid_0's rmse: 480.983
[1387]	valid_0's rmse: 480.931
[1388]	valid_0's rmse: 480.89
[1389]	valid_0's rmse: 480.871
[1390]	valid_0's rmse: 480.862
[1391]	valid_0's rmse: 480.831
[1392]	valid_0's rmse: 480.753
[1393]	valid_0's rmse: 480.747
[1394]	valid_0's rmse: 480.726
[1395]	valid_0's rmse: 480.72
[1396]	valid_0's rmse: 480.7
[1397]	valid_0's rmse: 480.696
[1398]	valid_0's rmse: 480.669
[1399]	valid_0's rmse: 480.665
[1400]	valid_0's rmse: 480.659
[1401]	valid_0's rmse: 480.606
[1402]	valid_0's rmse: 480.577
[1403]	valid_0's rmse: 480.51
[1404]	valid_0's rmse: 480.498
[1405]	valid_0's rmse: 480.494
[1406]	valid_0's rmse: 480.467
[1407]	valid_0'

[1641]	valid_0's rmse: 476.794
[1642]	valid_0's rmse: 476.792
[1643]	valid_0's rmse: 476.817
[1644]	valid_0's rmse: 476.819
[1645]	valid_0's rmse: 476.77
[1646]	valid_0's rmse: 476.768
[1647]	valid_0's rmse: 476.761
[1648]	valid_0's rmse: 476.737
[1649]	valid_0's rmse: 476.746
[1650]	valid_0's rmse: 476.73
[1651]	valid_0's rmse: 476.717
[1652]	valid_0's rmse: 476.712
[1653]	valid_0's rmse: 476.748
[1654]	valid_0's rmse: 476.738
[1655]	valid_0's rmse: 476.72
[1656]	valid_0's rmse: 476.719
[1657]	valid_0's rmse: 476.703
[1658]	valid_0's rmse: 476.666
[1659]	valid_0's rmse: 476.637
[1660]	valid_0's rmse: 476.615
[1661]	valid_0's rmse: 476.608
[1662]	valid_0's rmse: 476.605
[1663]	valid_0's rmse: 476.606
[1664]	valid_0's rmse: 476.653
[1665]	valid_0's rmse: 476.655
[1666]	valid_0's rmse: 476.654
[1667]	valid_0's rmse: 476.607
[1668]	valid_0's rmse: 476.602
[1669]	valid_0's rmse: 476.601
[1670]	valid_0's rmse: 476.581
[1671]	valid_0's rmse: 476.579
[1672]	valid_0's rmse: 476.569
[1673]	vali

### catboost

In [85]:
train_cb = cb.Pool(train_x, train_y)
params_cb = {'eval_metric': 'RMSE',
            'od_type': 'Iter',
             'iterations': 9999,
            'od_wait': 10,
            'use_best_model': True}
validation_cb = cb.Pool(test_x, test_y)

In [ ]:
%%time
model_cb = cb.train(train_cb, params_cb, eval_set=validation_cb)

0:	learn: 6808.5660624	test: 6825.7079329	best: 6825.7079329 (0)	total: 115ms	remaining: 19m 13s
1:	learn: 6615.4754021	test: 6636.0776687	best: 6636.0776687 (1)	total: 206ms	remaining: 17m 9s
2:	learn: 6428.0836387	test: 6452.5881408	best: 6452.5881408 (2)	total: 291ms	remaining: 16m 8s
3:	learn: 6246.5366468	test: 6274.8571380	best: 6274.8571380 (3)	total: 402ms	remaining: 16m 45s
4:	learn: 6070.4943726	test: 6102.1603195	best: 6102.1603195 (4)	total: 505ms	remaining: 16m 49s
5:	learn: 5900.2454261	test: 5935.5022809	best: 5935.5022809 (5)	total: 590ms	remaining: 16m 23s
6:	learn: 5735.1661839	test: 5773.3720112	best: 5773.3720112 (6)	total: 706ms	remaining: 16m 47s
7:	learn: 5574.9860627	test: 5616.6855791	best: 5616.6855791 (7)	total: 796ms	remaining: 16m 34s
8:	learn: 5420.1381133	test: 5464.7581336	best: 5464.7581336 (8)	total: 882ms	remaining: 16m 18s
9:	learn: 5270.2984390	test: 5317.7413482	best: 5317.7413482 (9)	total: 1.01s	remaining: 16m 53s
10:	learn: 5124.8402884	test: 51

85:	learn: 1263.1214632	test: 1341.6896620	best: 1341.6896620 (85)	total: 8.78s	remaining: 16m 51s
86:	learn: 1254.2025828	test: 1331.9379832	best: 1331.9379832 (86)	total: 8.87s	remaining: 16m 50s
87:	learn: 1245.9820548	test: 1322.5631146	best: 1322.5631146 (87)	total: 8.95s	remaining: 16m 48s
88:	learn: 1236.6830852	test: 1312.5966015	best: 1312.5966015 (88)	total: 9.06s	remaining: 16m 49s
89:	learn: 1229.1024685	test: 1303.8540759	best: 1303.8540759 (89)	total: 9.18s	remaining: 16m 50s
90:	learn: 1221.0984686	test: 1294.9375101	best: 1294.9375101 (90)	total: 9.28s	remaining: 16m 50s
91:	learn: 1213.7256309	test: 1286.6142491	best: 1286.6142491 (91)	total: 9.39s	remaining: 16m 50s
92:	learn: 1206.8479055	test: 1278.8035913	best: 1278.8035913 (92)	total: 9.47s	remaining: 16m 49s
93:	learn: 1200.1693274	test: 1271.3336964	best: 1271.3336964 (93)	total: 9.59s	remaining: 16m 50s
94:	learn: 1193.6861868	test: 1264.2224815	best: 1264.2224815 (94)	total: 9.7s	remaining: 16m 50s
95:	learn: 

168:	learn: 1003.5760725	test: 1043.2817849	best: 1043.2817849 (168)	total: 17.6s	remaining: 17m 1s
169:	learn: 1002.5285452	test: 1042.1279306	best: 1042.1279306 (169)	total: 17.7s	remaining: 17m 1s
170:	learn: 1001.5349368	test: 1041.0514535	best: 1041.0514535 (170)	total: 17.8s	remaining: 17m
171:	learn: 1000.5518303	test: 1039.5504958	best: 1039.5504958 (171)	total: 17.9s	remaining: 17m
172:	learn: 999.4791563	test: 1038.5093631	best: 1038.5093631 (172)	total: 18s	remaining: 16m 59s
173:	learn: 998.5688624	test: 1037.3736877	best: 1037.3736877 (173)	total: 18s	remaining: 16m 58s
174:	learn: 997.7467623	test: 1036.6984617	best: 1036.6984617 (174)	total: 18.2s	remaining: 16m 59s
175:	learn: 996.4422407	test: 1035.4509637	best: 1035.4509637 (175)	total: 18.2s	remaining: 16m 58s
176:	learn: 995.5991830	test: 1034.7110241	best: 1034.7110241 (176)	total: 18.3s	remaining: 16m 57s
177:	learn: 994.5396162	test: 1033.6241570	best: 1033.6241570 (177)	total: 18.4s	remaining: 16m 57s
178:	learn

252:	learn: 938.6041358	test: 974.0898189	best: 974.0898189 (252)	total: 26.5s	remaining: 16m 58s
253:	learn: 937.7244252	test: 973.1629024	best: 973.1629024 (253)	total: 26.6s	remaining: 16m 58s
254:	learn: 937.3705143	test: 972.8058607	best: 972.8058607 (254)	total: 26.7s	remaining: 16m 58s
255:	learn: 937.0322154	test: 972.5721997	best: 972.5721997 (255)	total: 26.8s	remaining: 16m 58s
256:	learn: 936.5869157	test: 972.0713494	best: 972.0713494 (256)	total: 26.9s	remaining: 16m 57s
257:	learn: 936.0624701	test: 971.6569653	best: 971.6569653 (257)	total: 27s	remaining: 16m 57s
258:	learn: 935.2350923	test: 970.7834237	best: 970.7834237 (258)	total: 27.1s	remaining: 16m 57s
259:	learn: 934.4498444	test: 970.0117422	best: 970.0117422 (259)	total: 27.2s	remaining: 16m 57s
260:	learn: 934.0374609	test: 969.4292135	best: 969.4292135 (260)	total: 27.3s	remaining: 16m 57s
261:	learn: 933.5954891	test: 968.9938131	best: 968.9938131 (261)	total: 27.4s	remaining: 16m 56s
262:	learn: 933.006739

338:	learn: 897.0141680	test: 931.6497696	best: 931.6497696 (338)	total: 35s	remaining: 16m 35s
339:	learn: 896.7162000	test: 931.3480645	best: 931.3480645 (339)	total: 35.1s	remaining: 16m 35s
340:	learn: 896.3615856	test: 931.0395061	best: 931.0395061 (340)	total: 35.1s	remaining: 16m 35s
341:	learn: 895.9328492	test: 930.6340454	best: 930.6340454 (341)	total: 35.3s	remaining: 16m 35s
342:	learn: 895.5291873	test: 930.2523069	best: 930.2523069 (342)	total: 35.3s	remaining: 16m 34s
343:	learn: 894.7856485	test: 929.5384941	best: 929.5384941 (343)	total: 35.4s	remaining: 16m 34s
344:	learn: 894.3794021	test: 929.1673514	best: 929.1673514 (344)	total: 35.6s	remaining: 16m 35s
345:	learn: 894.1053115	test: 928.8807018	best: 928.8807018 (345)	total: 35.6s	remaining: 16m 34s
346:	learn: 893.5118090	test: 928.0902185	best: 928.0902185 (346)	total: 35.7s	remaining: 16m 34s
347:	learn: 893.2665375	test: 927.8593095	best: 927.8593095 (347)	total: 35.8s	remaining: 16m 34s
348:	learn: 893.038623

...

4501:	learn: 508.7812744	test: 557.0366688	best: 557.0366688 (4501)	total: 7m 48s	remaining: 9m 32s
4502:	learn: 508.7772728	test: 557.0348040	best: 557.0348040 (4502)	total: 7m 48s	remaining: 9m 32s
4503:	learn: 508.7721751	test: 557.0318347	best: 557.0318347 (4503)	total: 7m 48s	remaining: 9m 32s
4504:	learn: 508.7599719	test: 557.0197434	best: 557.0197434 (4504)	total: 7m 48s	remaining: 9m 31s
4505:	learn: 508.7482330	test: 557.0072086	best: 557.0072086 (4505)	total: 7m 49s	remaining: 9m 31s
4506:	learn: 508.7300063	test: 556.9924003	best: 556.9924003 (4506)	total: 7m 49s	remaining: 9m 31s
4507:	learn: 508.7119391	test: 556.9789347	best: 556.9789347 (4507)	total: 7m 49s	remaining: 9m 31s
4508:	learn: 508.6928209	test: 556.9608333	best: 556.9608333 (4508)	total: 7m 49s	remaining: 9m 31s
4509:	learn: 508.6718585	test: 556.9366846	best: 556.9366846 (4509)	total: 7m 49s	remaining: 9m 31s
4510:	learn: 508.6661734	test: 556.9392319	best: 556.9366846 (4509)	total: 7m 49s	remaining: 9m 31s


4585:	learn: 507.2979028	test: 555.7863234	best: 555.7863234 (4585)	total: 7m 56s	remaining: 9m 22s
4586:	learn: 507.2914400	test: 555.7787425	best: 555.7787425 (4586)	total: 7m 56s	remaining: 9m 22s
4587:	learn: 507.2517853	test: 555.7372387	best: 555.7372387 (4587)	total: 7m 56s	remaining: 9m 22s
4588:	learn: 507.2460947	test: 555.7352353	best: 555.7352353 (4588)	total: 7m 56s	remaining: 9m 22s
4589:	learn: 507.2121998	test: 555.6853152	best: 555.6853152 (4589)	total: 7m 57s	remaining: 9m 22s
4590:	learn: 507.2044681	test: 555.6806924	best: 555.6806924 (4590)	total: 7m 57s	remaining: 9m 22s
4591:	learn: 507.1983898	test: 555.6792824	best: 555.6792824 (4591)	total: 7m 57s	remaining: 9m 21s
4592:	learn: 507.1916012	test: 555.6740975	best: 555.6740975 (4592)	total: 7m 57s	remaining: 9m 21s
4593:	learn: 507.1792402	test: 555.6625277	best: 555.6625277 (4593)	total: 7m 57s	remaining: 9m 21s
4594:	learn: 507.1700067	test: 555.6506562	best: 555.6506562 (4594)	total: 7m 57s	remaining: 9m 21s


4670:	learn: 505.9362457	test: 554.7462539	best: 554.7462539 (4670)	total: 8m 6s	remaining: 9m 15s
4671:	learn: 505.9302275	test: 554.7422767	best: 554.7422767 (4671)	total: 8m 6s	remaining: 9m 14s
4672:	learn: 505.9257505	test: 554.7375770	best: 554.7375770 (4672)	total: 8m 6s	remaining: 9m 14s
4673:	learn: 505.9190246	test: 554.7456207	best: 554.7375770 (4672)	total: 8m 6s	remaining: 9m 14s
4674:	learn: 505.9030757	test: 554.7359975	best: 554.7359975 (4674)	total: 8m 7s	remaining: 9m 14s
4675:	learn: 505.9010196	test: 554.7374509	best: 554.7359975 (4674)	total: 8m 7s	remaining: 9m 14s
4676:	learn: 505.8972187	test: 554.7420821	best: 554.7359975 (4674)	total: 8m 7s	remaining: 9m 14s
4677:	learn: 505.8711152	test: 554.7234669	best: 554.7234669 (4677)	total: 8m 7s	remaining: 9m 14s
4678:	learn: 505.8341929	test: 554.6945657	best: 554.6945657 (4678)	total: 8m 7s	remaining: 9m 14s
4679:	learn: 505.8135190	test: 554.6763279	best: 554.6763279 (4679)	total: 8m 7s	remaining: 9m 14s
4680:	lear

4753:	learn: 504.7377408	test: 553.9285597	best: 553.9285597 (4753)	total: 8m 14s	remaining: 9m 5s
4754:	learn: 504.7293922	test: 553.9245580	best: 553.9245580 (4754)	total: 8m 14s	remaining: 9m 5s
4755:	learn: 504.7086300	test: 553.9067125	best: 553.9067125 (4755)	total: 8m 14s	remaining: 9m 5s
4756:	learn: 504.6860796	test: 553.8930459	best: 553.8930459 (4756)	total: 8m 14s	remaining: 9m 5s
4757:	learn: 504.6545520	test: 553.8663795	best: 553.8663795 (4757)	total: 8m 14s	remaining: 9m 5s
4758:	learn: 504.6492845	test: 553.8668126	best: 553.8663795 (4757)	total: 8m 15s	remaining: 9m 5s
4759:	learn: 504.6405022	test: 553.8578384	best: 553.8578384 (4759)	total: 8m 15s	remaining: 9m 4s
4760:	learn: 504.6088455	test: 553.8291425	best: 553.8291425 (4760)	total: 8m 15s	remaining: 9m 4s
4761:	learn: 504.5898253	test: 553.8083461	best: 553.8083461 (4761)	total: 8m 15s	remaining: 9m 4s
4762:	learn: 504.5632113	test: 553.7773369	best: 553.7773369 (4762)	total: 8m 15s	remaining: 9m 4s
4763:	lear

4836:	learn: 503.1474773	test: 552.4461509	best: 552.4441178 (4834)	total: 8m 22s	remaining: 8m 56s
4837:	learn: 503.1380330	test: 552.4406456	best: 552.4406456 (4837)	total: 8m 22s	remaining: 8m 56s
4838:	learn: 503.1339613	test: 552.4387238	best: 552.4387238 (4838)	total: 8m 22s	remaining: 8m 56s
4839:	learn: 503.1080296	test: 552.4136381	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 56s
4840:	learn: 503.1031376	test: 552.4212417	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 56s
4841:	learn: 503.0969285	test: 552.4167458	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 55s
4842:	learn: 503.0921168	test: 552.4150456	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 55s
4843:	learn: 503.0854067	test: 552.4192844	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 55s
4844:	learn: 503.0829638	test: 552.4191061	best: 552.4136381 (4839)	total: 8m 23s	remaining: 8m 55s
4845:	learn: 503.0491207	test: 552.3796036	best: 552.3796036 (4845)	total: 8m 23s	remaining: 8m 55s
